# Proyecto procesamiento de imagenes

In [4]:
import cv2
from pyzbar.pyzbar import decode
import matplotlib.pyplot as plt

In [5]:
class Pipeline:
    def __init__(self, steps):
        self.steps = steps

    def run(self, data):
        for step in self.steps:
            data = step(data)
        return data

def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def threshold(threshold_value=128):
    def inner(image, threshold_value=threshold_value):
        _, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
        return thresholded_image
    return inner

In [14]:
# Decode QR codes
def decode_qr(qr_image):
  decoded_objects = decode(qr_image)
  if not decoded_objects:
      return False, ""
  return True, decoded_objects[0].data.decode("utf-8")


def get_txt_content(file_name):
  with open(file_name, 'r') as file:
    content = file.read()
  return content

In [15]:
def checkQr(image, txt_file_path):
  content = get_txt_content(txt_file_path)

  ok, decoded = decode_qr(image)
  if ok:
    return decoded == content
  return False



In [ ]:
# Get file names in /png
import os
files = os.listdir("png")
files = [file.split(".")[0] for file in files]

# Set pipeline
pipeline = Pipeline([
    # threshold(128),
    # grayscale
])

# Initialize counters to compare original vs transformed 
enhanced_count = 0
worsened_count = 0
same_ok_count = 0
same_false_count = 0


for file in files:
  # Read image
  img = cv2.imread("png/" + file + ".png")
  if img is None:
    print(f"Image {file}.png not found.")
    continue

  # Read text file
  file_name = "txt/" + file + ".txt"

  # Apply pipeline
  img_transformed = pipeline.run(img)

  # Show result of pipeline 
  ok = checkQr(img, file_name)
  transformed_ok  = checkQr(img_transformed, file_name)

  if ok:
    if transformed_ok: same_ok_count += 1
    else: worsened_count += 1
  else:
    if transformed_ok: enhanced_count += 1
    else: same_false_count += 1
  

# Print summary
print(f"Total images: {len(files)}")
print(f"enhanced_count: {enhanced_count}")
print(f"worsened_count: {worsened_count}")
print(f"same_ok_count: {same_ok_count}")
print(f"same_false_count: {same_false_count}")








	i=2 f=-1(000) part=1
	i=2 f=-1(000) part=1
	i=26 f=-1(000) part=0
	i=26 f=-1(000) part=0
	i=10 f=-1(000) part=0
	i=9 f=-1(000) part=1
	i=10 f=-1(000) part=0
	i=9 f=-1(000) part=1


Total images: 3332
enhanced_count: 0
worsened_count: 0
same_ok_count: 1840
same_false_count: 1492


In [ ]:
def opening(image):
    # Create a kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    # Perform opening operation
    opened_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    return opened_image

qr_image = cv2.imread("png/000004.png")
plt.imshow(qr_image)
plt.axis('off')
plt.show()
ok, decoded = decode_qr(qr_image)
if ok:
    print("Decoded QR Code:", decoded)
else:
    print("No QR Code found in the image.")
qr_image = grayscale(qr_image)
plt.imshow(qr_image, cmap='gray')
plt.axis('off')
plt.show()

qr_image = opening(qr_image)
plt.imshow(qr_image, cmap='gray')
plt.axis('off')
plt.show()
ok, decoded = decode_qr(qr_image)
if ok:
    print("Decoded QR Code:", decoded)
else:
    print("No QR Code found in the image.")